In [1]:
import pandas as pd
import numpy as np
import math
import collections
import re
import sys
import itertools

In [4]:

data = pd.read_csv('./input.txt',header=None,delim_whitespace=False)
lines = data.values
registers = {}

commands = [line[0].split(' ') for line in lines]
idx2 = 0
idx = 0
while idx < len(commands) and idx2 < 20:
    cmd = commands[idx]
   # print cmd
    jump = 0
    if cmd[0] == 'cpy':
        value = 0
        if cmd[1].isdigit():
            value = int(cmd[1])
        else:
            value = registers[cmd[1]]
        registers[cmd[2]] = value
    elif cmd[0] == 'inc':
        registers[cmd[1]] = registers[cmd[1]] + 1
    elif cmd[0] == 'dec':
        registers[cmd[1]] = registers[cmd[1]] - 1
    elif cmd[0] == 'jnz':
        if cmd[1] not in registers or registers[cmd[1]] != 0:
            jump = int(cmd[2])
            
    if jump == 0:
        idx += 1 
    else:
        idx += jump
        
    #print registers   
print registers

{'a': 42}


In [260]:
## generate map
map_size = 100
floorplan = np.zeros((map_size,map_size))

seed = 1364
#seed = 10
for y in range(map_size):
    for x in range(map_size):
        code = x*x + 3*x + 2*x*y + y + y*y + seed
        if bin(code).count("1") % 2 == 0:
            floorplan[x,y] = 0
        else:
            floorplan[x,y] = 1

In [261]:
def validmove(pos,floorplan,visited):
    if pos[0] >= map_size or pos[1] >= map_size:
        return False
    
    if pos[0] < 0  or pos[1] < 0:
        return False
    
    if floorplan[pos[0],pos[1]] == 0 and pos not in visited:
        return True
    else:
        return False

In [262]:
class node:
    def __init__(self,position,depth=0):
        self.position = position
        self.depth = depth
        
    def generateValidMoves(self,goal,visited):
        valid_moves = []
        ## check up 
        up = node((self.position[0],self.position[1]-1),self.depth+1)
        if validmove(up.position,floorplan,visited):
            valid_moves.append(up)
            
        down = node((self.position[0],self.position[1] + 1),self.depth+1)
        if validmove(down.position,floorplan,visited):
            valid_moves.append(down)
            
            
        left = node((self.position[0]-1,self.position[1]),self.depth+1)
        if validmove(left.position,floorplan,visited):
            valid_moves.append(left)
            
        right = node((self.position[0]+1,self.position[1]),self.depth+1)
        if validmove(right.position,floorplan,visited):
            valid_moves.append(right)
            
        return valid_moves
    
    def cost(self,goal):
        manhattan_dist = abs(self.position[0]-goal[0]) + abs(self.position[1]-goal[1])
        return manhattan_dist + self.position[0] -1 + self.position[1] - 1

In [267]:

from heapq import *

pq = []                         # list of entries arranged in a heap
entry_finder = {}               # mapping of tasks to entries
REMOVED = '<removed-task>'      # placeholder for a removed task
counter = itertools.count()     # unique sequence count

def add_task(task, priority=0):
    'Add a new task or update the priority of an existing task'
    if task in entry_finder:
        remove_task(task)
    count = next(counter)
    entry = [priority, count, task]
    entry_finder[task] = entry
    heappush(pq, entry)

def remove_task(task):
    'Mark an existing task as REMOVED.  Raise KeyError if not found.'
    entry = entry_finder.pop(task)
    entry[-1] = REMOVED

def pop_task():
    'Remove and return the lowest priority task. Raise KeyError if empty.'
    while pq:
        priority, count, task = heappop(pq)
        if task is not REMOVED:
            del entry_finder[task]
            return task
    raise KeyError('pop from an empty priority queue')
    

goal = (31,39)

root = node((1,1))
visited = []

add_task(root,root.cost(goal))
print "root "
print root.cost(goal)
goalreached = False
fifty_goals = []
while goalreached == False:
    currentnode = pop_task()
    moves = currentnode.generateValidMoves(goal,visited)
    
    for move in moves:
        if move.depth <= 50:
            print move.position
            add_task(move,move.cost(goal))
            
    if currentnode.position == goal:
        print "at goal, moves: " + str(currentnode.depth)
        break

    if currentnode.position not in visited:
        visited.append(currentnode.position)
    print len(visited)

root 
68
(0, 1)
1
(0, 2)
2
(0, 3)
3
(1, 3)
4
(1, 4)
5
(2, 4)
6
(2, 5)
7
(2, 6)
8
(2, 7)
(1, 6)
(3, 6)
9
(2, 8)
10
(0, 6)
11
(4, 6)
12
(3, 8)
13
(0, 5)
14
(5, 6)
15
(3, 9)
16
17
(5, 5)
(5, 7)
(6, 6)
18
(4, 9)
19
(5, 4)
(6, 5)
20
(5, 8)
21
(6, 5)
(7, 6)
22
(4, 10)
(5, 9)
23
(5, 3)
24
25
(5, 9)
26
26
(8, 6)
27
(5, 10)
28
(5, 10)
(6, 9)
29
(5, 2)
30
(5, 10)
(6, 9)
30
(9, 6)
31
32
32
(7, 9)
33
(5, 1)
34
34
(7, 9)
34
(10, 6)
35
(8, 9)
36
37
(8, 9)
37
(10, 7)
(11, 6)
38
(8, 8)
39
(8, 8)
39
(10, 8)
(11, 7)
40
(11, 7)
41
42
42
(10, 9)
43
(12, 7)
44
(12, 7)
44
(11, 9)
45
(12, 8)
46
(12, 8)
46
(11, 10)
47
48
48
(11, 11)
49
(11, 12)
(12, 11)
50
(11, 13)
(10, 12)
(12, 12)
51
(12, 12)
52
(11, 14)
53
(9, 12)
54
(13, 12)
55
(13, 12)
55
(12, 14)
56
(8, 12)
57
(13, 13)
58
(13, 13)
58
(13, 14)
59
(8, 13)
(7, 12)
60
(13, 14)
(14, 13)
61
(13, 14)
(14, 13)
61
62
(8, 14)
63
(7, 11)
64
64
(15, 13)
65
65
(15, 13)
65
(8, 15)
66
67
(16, 13)
68
(16, 13)
68
(8, 16)
69
(17, 13)
70
(17, 13)
70
(7, 16)
71
(17, 12)
(1

KeyError: 'pop from an empty priority queue'